In [1]:
from sage.symbolic.integration.integral import definite_integral, indefinite_integral
from sage.ext.fast_eval import fast_float

var('rho,t,x,y,Z,Z1,Z2')
rho = 0.67
sigma = sqrt(1.0-rho^2)
normal = RealDistribution('gaussian', 1.0)

def cumsum(x, Z):
    return definite_integral((1.0/sqrt(2.0*pi))*exp(-t^2/2.9), t, -infinity, (2.0*Z-x-rho*x)/sigma)

y_pdf = (1.0/(sigma*sqrt(2.0*pi)))*exp(-0.5*((y-rho*x)/sigma)^2)
#numerical_approx(cumsum(0,0,3*sigma*1/2))
y_pdf

0.952510554470738*e^(-0.500000000000000*(-0.902525740772143*x + 1.34705334443603*y)^2)/sqrt(pi)

In [2]:
#(y*y_pdf).integral(y, 2-x, 4-x) this gives an error
x_pdf = 1/sqrt(2*pi)*e^(-x^2/2)
x_pdf

1/2*sqrt(2)*e^(-1/2*x^2)/sqrt(pi)

In [3]:
integrand = (x*y*y_pdf*e^(-x^2/2.00)/sqrt(2.0*pi)).simplify()
integrand

0.6735266722180172*x*y*e^(-0.5*x^2 - 0.5*(-0.9025257407721431*x + 1.347053344436034*y)^2)/pi

In [4]:
#Z1 = -4.0
#Z2 = 4.0
inc = 0.01


def calc_moments(Z1, Z2):
    X = -5.0
    first_moment = 0.0
    second_moment = 0.0
    total_cum_density = 0.0
    while X <= 5:
        y_cum_density = normal.cum_distribution_function((2.0*Z2-X-rho*X)/sigma)-normal.cum_distribution_function((2.0*Z1-X-rho*X)/sigma)
        try:
            total_cum_density += inc*numerical_approx(x_pdf.subs(x==X)*y_cum_density)
            first_moment += inc*numerical_approx(X*x_pdf.subs(x==X)*y_cum_density)
            second_moment += inc*numerical_approx(X^2*x_pdf.subs(x==X)*y_cum_density)
        except:
            pass
        X += inc

    print("total cumulative density: " + str(total_cum_density))
    print("expected: " + str(first_moment/total_cum_density))
    print("expected2: " + str(second_moment/total_cum_density))
    return total_cum_density, first_moment, second_moment

In [5]:
def calc_xy_moment(Z1, Z2):
    X = -5
    integral = 0.0
    ct = 0
    while X <= 5:
        X += inc
        Y = 2*Z1-X
        if X - Y >= 4:
            Y = X - 4
        while Y <= 2.0*Z2-X:
            if Y > 5 or Y-X >= 4:
                break
            try:
                value_at_point = numerical_approx(integrand.subs(x=X,y=Y))
                integral += value_at_point*inc^2
                ct += 1
                #if ct % 10000 == 0:
                #    print(ct)
                #    print(str(X) + " " + str(Y))
            except:
                print("exception at: " + str(X) + " " + str(Y))
            Y += inc

    return numerical_approx(integral)

In [6]:
def calc_correlation(Z1, Z2):
    print("midpoint between: " + str(Z1)+" "+str(Z2))
    total_cum_density, first_moment, second_moment = calc_moments(Z1, Z2)
    expected_xy = calc_xy_moment(Z1, Z2)/total_cum_density
    print("expected xy: " + str(expected_xy))
    expected = first_moment/total_cum_density
    expected2 = second_moment/total_cum_density
    covariance = expected_xy - expected^2
    variance = expected2 - expected^2
    print("covariance: " + str(covariance))
    print("std: " + str(sqrt(variance)))
    corr = covariance / variance
    print("correlation: " + str(corr))

In [7]:
calc_correlation(1, 2)

midpoint between: 1 2
total cumulative density: 0.122591234088536
expected: 1.36288319574211
expected2: 2.09182185469235
expected xy: 1.76564403956961
covariance: -0.0918065656666227
std: 0.484119044715364
correlation: -0.391714281848433


In [8]:
calc_correlation(-4, 4)

midpoint between: -4 4
total cumulative density: 0.999987877427826
expected: -1.79054354354497e-16
expected2: 0.999795814267914
expected xy: 0.669801891467957
covariance: 0.669801891467957
std: 0.999897901921948
correlation: 0.669938683388478


In [9]:
calc_correlation(3, 4)

midpoint between: 3 4
total cumulative density: 0.000507383886533620
expected: 3.23008341987944
expected2: 10.6395536280096
expected xy: 10.3989336280786
covariance: -0.0345052713014571
std: 0.453998599810123
correlation: -0.167408081561570
